In [ ]:
from pib import PIB
import pandas as pd

In [ ]:
pib = PIB('490dcd21-44b2-fef6-0ade-e6431a8c8fb9')

entidades = (pd.read_parquet("../../data/catalogos/estados/estados.parquet")
            #  .sort_values(by="nom_agee")
             ).nom_agee.apply(lambda x: x.capitalize())

In [ ]:
def pib_real():
    df = pd.DataFrame()

    for entidad in entidades:
        print(entidad)
        temp = pib.get_data_estatal(grupo = "estado", entidad=entidad)
        temp["entidad"] = entidad
        df = pd.concat([df,temp])

    return df

df_ = pib_real()

In [ ]:
def pib_nom():
    df = pd.DataFrame()

    for entidad in entidades:
        print(entidad)
        temp = pib.get_data_estatal(indicador="PIB anual a precios corrientes", grupo = "estado", entidad=entidad)
        temp["entidad"] = entidad
        df = pd.concat([df,temp])

    return df

df_nom = pib_nom()

In [ ]:
df_.to_excel("../../data/pib/pib_real.xlsx")
df_nom.to_excel("../../data/pib/pib_nominal.xlsx")

In [ ]:
df_ = pd.read_excel("../../data/pib/pib_real.xlsx", dtype={"periodo":str}).set_index("periodo")

df_nom = pd.read_excel("../../data/pib/pib_nominal.xlsx", dtype={"periodo":str}).set_index("periodo")

In [ ]:
def ranking(df, sector, year:int):
    ranking = pd.DataFrame()
    year = str(year)
    for entidad in entidades:
        temp = df[df["entidad"] == entidad].loc[[year]]
        temp["entidad"] = entidad
        ranking = pd.concat([ranking,temp])
    
    ranking = ranking.reset_index(drop=True)
    ranking = ranking.set_index("entidad")
    ranking = ranking[[sector]]
    ranking["rank"] = (ranking.rank(method="first", ascending=False))
    ranking = ranking.sort_values(by="rank")

    X = ranking[sector].sum()

    ranking["% nacional"] = (ranking[sector]/X)*100

    return ranking

ranking(df_nom, "Total industria manufacturera", 2022)

In [169]:
def variacion(df, sector:str, year: int):
    df_f = pd.DataFrame()

    yeari = str(year - 1)
    yearf = str(year)

    for entidad in entidades:
        temp = df[df["entidad"] == entidad].loc[[yeari, yearf]]
        temp["entidad"] = entidad
        temp = temp[["entidad",sector]]
        temp["cambio %"] = (temp[sector].pct_change())*100
        # temp = temp.dropna()
        df_f = pd.concat([df_f,temp])
    

    return df_f

variacion(df_, "Total industria manufacturera", 2022)

,entidad,Total industria manufacturera,cambio %
periodo,,,
2021,Baja california,330405.218,NaN
2022,Baja california,342887.949,3.778007
2021,Baja california sur,3935.328,NaN
2022,Baja california sur,4027.591,2.344481
2021,Campeche,4221.670,NaN
...,...,...,...
2022,Tamaulipas,246568.446,6.590693
2021,Tlaxcala,40845.040,NaN
2022,Tlaxcala,42138.355,3.166394


In [173]:
def  var_nac():
    df = (variacion(df_, "Total industria manufacturera", 2022)
     .reset_index())
    
    sum_2021 = df[df["periodo"] == "2021"]["Total industria manufacturera"].sum()
    sum_2022 = df[df["periodo"] == "2022"]["Total industria manufacturera"].sum()

    var = ((sum_2022 - sum_2021)/sum_2021)*100

    return var

var_nac()

6.335785292792581

In [119]:
class INFO_PIB:

    def __init__(self) -> None:
        pass


    def _ranking(self, df, sector, year):
        ranking = pd.DataFrame()
        year = str(year)
        for entidad in entidades:
            temp = df[df["entidad"] == entidad].loc[[year]]
            temp["entidad"] = entidad
            ranking = pd.concat([ranking,temp])
        
        ranking = ranking.reset_index(drop=True)
        ranking = ranking.set_index("entidad")
        ranking = ranking[[sector]]
        ranking["rank"] = (ranking.rank(method="first", ascending=False))
        ranking = ranking.sort_values(by="rank")

        X = ranking[sector].sum()

        ranking["% nacional"] = (ranking[sector]/X)*100

        return ranking
    

    def _variacion(self, df, sector, year):
        df_f = pd.DataFrame()
        yeari = str(year - 1)
        yearf = str(year)

        for entidad in entidades:
            temp = df[df["entidad"] == entidad].loc[[yeari, yearf]]
            temp["entidad"] = entidad
            temp = temp[["entidad",sector]]
            temp["cambio %"] = (temp[sector].pct_change())*100
            temp = temp.dropna()
            df_f = pd.concat([df_f,temp])
        

        return df_f
    
    
    def get_ranking(self,
                    df: pd.DataFrame,
                    sector: str|list = "Total", 
                    year: int = 2022,
                    estado: str = "Aguascalientes"
                    ):
        
        ranking = self._ranking(df, sector, year)
        var = ranking.loc[estado][0]
        rank = ranking.loc[estado][1]
        nac =  round(ranking.loc[estado][2],2)

        return [var, rank, nac]
    

    def get_var(self, 
                df: pd.DataFrame,
                sector: str = "Total",
                year: int = 2022,
                estado: str = "Aguascalientes",
                ):
        
        var = self._variacion(df, sector, year).set_index("entidad")
        var = round(var.loc[estado][1], 2)

        return var


info = INFO_PIB()

In [120]:
info.get_var(df_)

1.13